In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [221]:
# SexとEmbarkedを数値に置き換えて読み込む
train_data = pd.read_csv("train.csv").replace({"male":1,"female":0}).replace({"S":0,"C":1,"Q":2})
test_data = pd.read_csv("test.csv").replace({"male":1,"female":0}).replace({"S":0,"C":1,"Q":2})

In [222]:
def preprocess(df):
    
    df["title"] = df.Name.str.extract('([A-Za-z]+)\.')
    
    # 敬称をMiss ,Mr,Mrs,Master,Otherの5分類にまとめる
    df['title'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don','Dona'],\
                            ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr','Mr'],inplace=True)
    
    df["Age"] = train_data[["title","Age"]].groupby("title")["Age"].apply(lambda x:x.fillna(x.mean()))
    
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master":4,"Other":5,} 
    df['title'] = df['title'].map(title_mapping) 

    df["has_Cabin"] = 1
    df["has_Cabin"][df["Cabin"].isnull()] =0
    
    # Embarkedの補完
    df["Embarked"].fillna(1.0,inplace=True)
    
    # Fareの補完
    mean = df[(df["Pclass"]==3) &(df["Embarked"]==0) &(df["Parch"]==0)]["Fare"].mean()
    df["Fare"].fillna(mean,inplace=True)

    # PassengerId / Name / Ticket / Cabinを削除する
    df = df.drop(["PassengerId","Name","Ticket","Cabin"],axis=1)
    return df

In [223]:
train = preprocess(train_data)
test = preprocess(test_data)

/Users/hiroki.kurasawa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/hiroki.kurasawa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [224]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,title,has_Cabin
0,0,3,1,22.0,1,0,7.2500,0.0,1,0
1,1,1,0,38.0,1,0,71.2833,1.0,3,1
2,1,3,0,26.0,0,0,7.9250,0.0,2,0
3,1,1,0,35.0,1,0,53.1000,0.0,3,1
4,0,3,1,35.0,0,0,8.0500,0.0,1,0


In [225]:
test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,title,has_Cabin
0,3,1,22.0,0,0,7.8292,2,1,0
1,3,0,38.0,1,0,7.0000,0,3,0
2,2,1,26.0,0,0,9.6875,2,1,0
3,3,1,35.0,0,0,8.6625,0,1,0
4,3,0,35.0,1,1,12.2875,0,3,0


In [226]:
train_x = train.drop("Survived",axis=1)
train_y = train.Survived

In [227]:
# 検証用データの作成
from sklearn.model_selection  import train_test_split

(train_x,test_x,train_y,test_y) = train_test_split(train_x,train_y,test_size =0.4,random_state=42)
print("train_x size >>{}".format(train_x.shape))
print("train_y size >>{}".format(train_y.shape))
print("test_x size >>{}".format(test_x.shape))
print("test_y size >>{}".format(test_y.shape))

train_x size >>(534, 9)
train_y size >>(534,)
test_x size >>(357, 9)
test_y size >>(357,)


In [142]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn import grid_search
from sklearn.metrics import accuracy_score

/Users/hiroki.kurasawa/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/hiroki.kurasawa/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [228]:
params = {
        'n_estimators'      : [10,25,50,75,100],
        'random_state'      : [0],
        'n_jobs'            : [4],
        'min_samples_split' : [5,10, 15, 20,25, 30],
        'max_depth'         : [5, 10, 15,20,25,30]
}

clf_rf = grid_search.GridSearchCV(RandomForestClassifier(random_state=0,n_jobs=-1),params,scoring='accuracy')
clf_rf.fit(train_x,train_y)
print("best_param:\n{}".format(clf_rf.best_params_))
print("best_score:\n{}".format(clf_rf.best_score_))

best_param:
{'max_depth': 15, 'min_samples_split': 10, 'n_estimators': 50, 'n_jobs': 4, 'random_state': 0}
best_score:
0.8426966292134831


In [230]:
clf_rf_best = RandomForestClassifier(n_estimators=50,max_depth=15,n_jobs=4,random_state=0,min_samples_split= 10)
clf_rf_best.fit(train_x,train_y)
y_test_pred = clf_rf_best.predict(test_x)
print("accuracy:{}".format(accuracy_score(y_test_pred,test_y)))

accuracy:0.8263305322128851


In [231]:
fti = clf_rf_best.feature_importances_  
for idx , feature in enumerate(test_x.columns):
    print("{0:10s}     >> {1:>.6f}".format(feature,fti[idx]))

Pclass         >> 0.107673
Sex            >> 0.202203
Age            >> 0.131442
SibSp          >> 0.040949
Parch          >> 0.032869
Fare           >> 0.172769
Embarked       >> 0.038074
title          >> 0.243217
has_Cabin      >> 0.030803


In [232]:
answer = clf_rf_best.predict(test)
submit_data = pd.Series(answer,name="Survived",index=test_data["PassengerId"])

In [233]:
submit_data.to_csv("submit_180820.csv",header=True)